<a href="https://colab.research.google.com/github/bmeelnlga/myportfolio/blob/main/UDN%E7%88%AC%E8%9F%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import requests
from bs4 import BeautifulSoup
import time
import json

class UDN:

  def __init__(self, page: int, start_url: str, start_id: int) -> None:
      self.page = page  # 要爬取的文章數
      self.start_url = start_url  # 基礎 URL
      self.start_id = start_id  # 開始的文章 ID
      self.article_list = []  # 存儲爬取的文章

  def fetch_data(self, url: str):
      """抓取網頁數據並返回未處理的 BeautifulSoup 對象"""
      try:
          response = requests.get(url)
          response.raise_for_status()  # 檢查請求是否成功
          soup = BeautifulSoup(response.text, 'html.parser')
          return soup
      except requests.RequestException as e:
          print(f"Error fetching data from {url}: {e}")
          return None

  def get_content(self, soup: BeautifulSoup):
      """提取文章的正文內容並去除 \n 和 \r"""
      contents = soup.findAll('p', class_=None)
      if contents:
          result = " ".join(content.text.replace('\n', '').replace('\r', '').strip() for content in contents)
          return result
      return None

  def get_datetime(self, soup: BeautifulSoup):
      """提取發佈日期"""
      datetime_tag = soup.find(class_='article-content__time')
      return datetime_tag.text.split(" ")[0] if datetime_tag else None

  def get_category(self, soup: BeautifulSoup):
      """提取文章的類別"""
      category_tag = soup.find('meta', {'property': 'article:section'})  # 根據網站的標籤結構進行調整
      return category_tag['content'] if category_tag else None

  def get_subtitle(self, soup: BeautifulSoup):
      """取文章的副標題 (如: 金融要聞)"""
      # 查找類名為 'breadcrumb-items' 且沒有 href 屬性的 <a> 標籤
      subtitle_tag = soup.find('a', class_='breadcrumb-items', href=False)

  # 返回副標題文字，如果找不到則返回 None
      return subtitle_tag.text.strip() if subtitle_tag else None


  def get_title(self, soup: BeautifulSoup):
      """提取文章標題"""
      title_tag = soup.find('h1')
      return title_tag.text.strip() if title_tag else None

  def get_comments(self, soup: BeautifulSoup):
      """提取留言數量"""
      comments_tag = soup.find(class_='article-comments')
      return comments_tag.text.strip().split(" ")[0] if comments_tag else "0"

  def get_info(self):
      """循環抓取文章信息並儲存"""
      for i in range(self.start_id - self.page, self.start_id):
          article_url = self.start_url + str(i)
          soup = self.fetch_data(article_url)

          if soup:
              category=self.get_category(soup)
              title= self.get_title(soup)
              datetime= self.get_datetime(soup)
              link=article_url
              content= self.get_content(soup)
              comments= self.get_comments(soup)
              subtitle= self.get_subtitle(soup)
              if title is None or content is None:
                  print(f"Skipping article ID {i} due to missing title or content.")
                  continue  # 跳過這篇文章

              article_content = {
                  "category": category,
                  "subtitle": subtitle,
                  "title": title,
                  "datetime": datetime,
                  "link": link,
                  "content": content,
                  "comments": comments,
              }

              self.article_list.append(article_content)  # 儲存文章信息
              print(json.dumps(article_content, ensure_ascii=False, indent=4))  # 打印 JSON 格式的文章信息

          time.sleep(0.3)  # 延時避免過快爬取

      # 儲存所有文章信息到 JSON 文件
      with open("udn_articles.json", "w", encoding='utf-8') as f:
          json.dump(self.article_list, f, ensure_ascii=False, indent=4)

udn = UDN(10, "https://udn.com/news/story/124222/", 8716497) # 測試爬取10篇文章
udn.get_info()

<ipython-input-7-c2576632db29>:27: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  contents = soup.findAll('p', class_=None)


{
    "category": "生活",
    "subtitle": "生活新聞",
    "title": "首批「萊豬」輸台恐引一波食安風暴？ 食藥署最新說明",
    "datetime": "2025-05-04",
    "link": "https://udn.com/news/story/124222/8716487",
    "content": "根據衛福部食藥署「豬肉儀表板」顯示，4月29日有一批從澳洲進口的22.99公噸豬其他可食部位，被檢出還有0.001PPM萊克多巴胺，引發各界擔憂，是否會有食安風險？食藥署表示，這批產品的萊劑殘留容許量在0.01PPM內，屬於可接受風險。  食藥署指出，近期食藥署邊境受理報驗澳洲「冷凍豬腳」1批（淨重2萬2991公斤），該批貨品依規定查驗並抽樣送驗乙型受體素，檢出萊克多巴胺0.001 PPM，符合「動物用藥殘留標準」於該類產品之殘留容許量0.01PPM，已於4月29日依規定核發輸入許可通知，並顯示於「豬肉儀表板」，所有合格放行案件，都在科學分析下可接受風險。  這是台灣開放萊豬進口後，首批含有萊克多巴胺的進口豬肉，食藥署強調，這一批肉品輸入產品符合「動物用藥殘留標準」，經輸入查驗合格放行，得於國內銷售流通。目前如此，未來也持續依「食安優先、科學分析、國際標準及市場稽查」辦理。  政府對於進口豬肉（含萊劑豬肉）於110年1月起，針對可輸入之生鮮冷藏冷凍豬肉產品，不分國別，採逐批查驗措施，經查驗符合規定始得輸入。食藥署強調，自110年1月1日起輸入豬肉產品，採逐批監視查驗，並持續依風險滾動調整查驗方式，現行各國輸入之豬肉產品，採一般抽批查驗。 ▪ 手機報稅超簡單！4步驟輕鬆搞定 一圖看節稅祕訣不卡關 根據衛福部食藥署「豬肉儀表板」顯示，4月29日有一批從澳洲進口的22.99公噸豬其他可食部位，被檢出還有0.001PPM... 4月29日一批22.99公噸自澳洲進口的豬其他可食部位，檢出0.001PPM萊克多巴胺，這也是台灣開放萊豬進口後，首批含... 周五晚上，北美館前豎起一根根閃亮的鋁管，不斷噴灑霧氣，彷彿要為即將進入夏季的城市降溫。這是北美館第12屆「X-site計... 為降低行人死亡，交通部今年祭出改善道安三支箭，其中包含改善路口照明，以「路段黃光、路口白光」為更換原則，選定全台7處行